
### Model Selection Rationale

In the context of transfer learning for biomedical optical imaging, specifically fine-tuning on chest X-ray datasets, the choice of pre-trained models is crucial for achieving high accuracy and efficient computation. Two models stand out in the `torchvision.models` library for this purpose: ResNet50 and MobileNetV2.

**ResNet50**: Known for its deep residual learning framework, it addresses the vanishing gradient problem allowing models to be substantially deeper with improved performance. Its "skip connections" facilitate the training of much deeper networks by allowing gradients to flow through the network. For tasks requiring high accuracy, ResNet50 is often a go-to model because of its proven track record in image recognition challenges.

**MobileNetV2**: Designed for mobile and embedded vision applications, it employs an inverted residual structure with bottleneck layers. It is optimized for speed and efficiency while maintaining reasonable accuracy, making it suitable for applications where computational resources are limited.

In summary, ResNet50 is selected for its high accuracy and MobileNetV2 for its balance between efficiency and performance. The computational time for fine-tuning these models on transfer learning tasks is also a significant factor, with ResNet50 requiring more resources due to its complexity, and MobileNetV2 being more resource-friendly.


In [17]:
import torchvision.models as models

# List all available models in torchvision
model_names = [name for name in dir(models) if not name.startswith('_')]

# Filter out only the models that have pre-trained versions available
pretrained_models = [name for name in model_names if 'pretrained' in dir(getattr(models, name))]

pretrained_models

[]

In [ ]:
# Selecting two models: one for high accuracy and one for efficiency and speed
# For high accuracy, we often choose models like ResNet or DenseNet
# For efficiency and speed, we might choose models like MobileNet or SqueezeNet

# For this example, let's select ResNet (known for high accuracy) and MobileNet (known for efficiency)

selected_models = {
    'resnet': getattr(models, 'resnet50')(pretrained=True),
    'mobilenet': getattr(models, 'mobilenet_v2')(pretrained=True)
}

# Save the names and the model objects in a dictionary for later use
models_info = {name: model for name, model in selected_models.items()}

models_info

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 65.9MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed 

{'resnet': ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): Bottleneck(
       (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (downsample): Sequential(
         (0): Conv2d(64, 256, kerne

In [18]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader


In [ ]:

# Assuming that 'data_dir' is the directory that contains the 'train' and 'val' folders of the dataset
data_dir = 'path_to_your_data'
batch_size = 32
num_classes = 2  # For example, normal and pneumonia

# Define transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load data
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}


NameError: ignored

In [ ]:

# Load a pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Replace the last fully connected layer with a new one that matches the number of classes in the new dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Transfer the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define number of epochs
num_epochs = 25



## Fine-tuning Process and Effectiveness

Fine-tuning involves adjusting the pre-trained models to the new dataset, which in this case is a chest X-ray dataset with two classes. The process includes replacing the last layer of the models to fit the number of classes and training the models using the defined loss function and optimizer.

### Effectiveness Analysis
After training, the models' performance is assessed based on their accuracy and loss on both the training and validation datasets. This provides insights into how well the models have adapted to the new data and whether there are signs of overfitting or underfitting.

For a comprehensive analysis, we should also consider:
- Training and validation curves over epochs to identify trends.
- Possible class imbalances that could affect performance metrics.
- The need for further hyperparameter tuning or data augmentation techniques to improve results.


In [ ]:

# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    print('Training complete')
    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)


Epoch 0/24
----------


NameError: ignored

In [ ]:

# Load a pre-trained MobileNetV2 model
mobile_model = models.mobilenet_v2(pretrained=True)

# Replace the classifier with a new one that matches the number of classes in the new dataset
mobile_model.classifier[1] = nn.Linear(mobile_model.last_channel, num_classes)

# Transfer the model to GPU if available
mobile_model = mobile_model.to(device)

# Define loss function
mobile_criterion = nn.CrossEntropyLoss()

# Define optimizer for MobileNetV2
mobile_optimizer = optim.SGD(mobile_model.parameters(), lr=0.001, momentum=0.9)

# Train the MobileNetV2 model
mobile_model = train_model(mobile_model, mobile_criterion, mobile_optimizer, num_epochs=num_epochs)


Epoch 0/24
----------


NameError: ignored

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader


In [ ]:

# Assuming that 'data_dir' is the directory that contains the 'train' and 'val' folders of the dataset
data_dir = 'path_to_your_data'
batch_size = 32
num_classes = 2  # For example, normal and pneumonia

# Define transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load data
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}


NameError: ignored

In [ ]:

# Load a pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers in the network
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a new one that matches the number of classes in the new dataset
# Only the parameters of the last layer will be updated during training
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Transfer the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer, but only optimize the last layer's parameters
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Define number of epochs
num_epochs = 25



## Converting Models into Fixed Feature Extractors

In this task, we convert pre-trained models into fixed feature extractors by freezing all layers except the last one. This approach allows us to leverage the learned features from the large datasets the models were originally trained on, while only fine-tuning the output layer to our specific task.

### Performance Assessment
The performance of fixed feature extractors is evaluated based on their accuracy on the validation dataset. Since only the last layer is trained, the process is typically faster and requires less computational resources compared to full model fine-tuning.

It's important to note that while fixed feature extractors can be very effective, their performance may also depend on how similar the new task is to the original task the model was trained on.


In [ ]:

# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    print('Training complete')
    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=num_epochs)


Epoch 0/24
----------


NameError: ignored

In [ ]:

# Load a pre-trained MobileNetV2 model
mobile_model = models.mobilenet_v2(pretrained=True)

# Freeze all layers in the network
for param in mobile_model.parameters():
    param.requires_grad = False

# Replace the classifier with a new one, only the last layer's parameters will be updated during training
mobile_model.classifier[1] = nn.Linear(mobile_model.last_channel, num_classes)

# Transfer the model to GPU if available
mobile_model = mobile_model.to(device)

# Define loss function for MobileNetV2
mobile_criterion = nn.CrossEntropyLoss()

# Define optimizer for MobileNetV2, but only optimize the last layer's parameters
mobile_optimizer = optim.SGD(mobile_model.classifier[1].parameters(), lr=0.001, momentum=0.9)

# Train the MobileNetV2 model
mobile_model = train_model(mobile_model, mobile_criterion, mobile_optimizer, num_epochs=num_epochs)


Epoch 0/24
----------


NameError: ignored


## Comparison and Analysis between Fine-Tuning and Feature Extraction

The comparison between the fine-tuning approach (Task B) and the use of ConvNets as fixed feature extractors (Task C) highlights the trade-offs between adaptability and computational efficiency.

### Fine-Tuning
In fine-tuning, the entire network's weights are updated, allowing the model to adjust more finely to the specifics of the new dataset. This often results in higher accuracy but at the cost of increased training time and computational resources.

### Fixed Feature Extraction
Conversely, using ConvNets as fixed feature extractors speeds up training since only the last layer's weights are updated. This can be more efficient but may yield lower accuracy if the pre-trained features are not as applicable to the new task.

### Analysis
The actual results will depend on the specific characteristics of the datasets and the nature of the tasks. For instance, if the new task is significantly different from the original training data, fine-tuning may provide substantial benefits. However, if the tasks are similar, feature extraction might suffice with much less computational effort.


In [ ]:

# Code to load actual results from Task B and Task C would go here.
# For the purposes of this notebook, we will use the placeholder dictionaries as an example.

results_task_b = {
    'resnet': {'train_acc': 0.95, 'val_acc': 0.85},
    'mobilenet': {'train_acc': 0.90, 'val_acc': 0.80}
}

results_task_c = {
    'resnet': {'train_acc': 0.90, 'val_acc': 0.80},
    'mobilenet': {'train_acc': 0.85, 'val_acc': 0.75}
}

def compare_results(results_task_b, results_task_c):
    for model_name in results_task_b.keys():
        print(f"Model: {model_name}")
        b_train_acc = results_task_b[model_name]['train_acc']
        b_val_acc = results_task_b[model_name]['val_acc']
        c_train_acc = results_task_c[model_name]['train_acc']
        c_val_acc = results_task_c[model_name]['val_acc']

        print(f"Fine-tuning - Training Accuracy: {b_train_acc}, Validation Accuracy: {b_val_acc}")
        print(f"Feature Extraction - Training Accuracy: {c_train_acc}, Validation Accuracy: {c_val_acc}")
        print()

# Run the comparison function
compare_results(results_task_b, results_task_c)



## Test Dataset Performance Analysis

Improving the performance on the test dataset can be challenging due to various factors such as overfitting, data imbalance, or the inherent difficulty of the task. This analysis aims to identify potential areas for improvement by examining the test results.

### Challenges
- **Overfitting**: The model may perform well on the training data but fails to generalize to unseen data.
- **Data Imbalance**: Certain classes may be underrepresented, leading to poor performance on those classes.
- **Data Quality**: Issues with the data itself, such as noise or errors in labeling, can adversely affect performance.
- **Model Capacity**: The model might be too simple to capture the complexity of the data or too complex, leading to overfitting.

### Analysis Process
- Review the confusion matrix to understand the misclassifications.
- Consider class-specific performance to identify if certain classes are problematic.
- Examine the errors made by the model to look for patterns or common issues.



# Code to load actual test results from Task B and Task C would go here.
# For the purposes of this notebook, we will use the placeholder dictionary as an example.

test_results = {
    'resnet': {'test_acc': 0.80},
    'mobilenet': {'test_acc': 0.75}
}

def analyze_test_performance(test_results):
    # Placeholder for the analysis process.
    for model_name, metrics in test_results.items():
        test_acc = metrics['test_acc']
        print(f"Model: {model_name}, Test Accuracy: {test_acc}")
        # Here you would add your analysis code, which might involve examining the errors made by the model,
        # looking at misclassified examples, or calculating additional performance metrics.

# Run the analysis function
analyze_test_performance(test_results)


In [ ]:

# Placeholder code for test dataset performance analysis
# In a real-world scenario, this code would load the test dataset results,
# perform the analysis, and potentially make use of libraries like scikit-learn for metrics and confusion matrix.

test_results = {'resnet': {'test_acc': 0.80}, 'mobilenet': {'test_acc': 0.75}}

def analyze_test_performance(test_results):
    for model_name, metrics in test_results.items():
        test_accuracy = metrics['test_acc']
        print(f"Model: {model_name}, Test Accuracy: {test_accuracy}")
        # Placeholder for the analysis process. This might include:
        # - Calculating and reviewing the confusion matrix.
        # - Checking for overfitting/underfitting using training/validation curves.
        # - Considering class imbalance and its impact on metrics.
        # - Additional preprocessing or data augmentation techniques.
        # - Exploring different model architectures or hyperparameter tuning.
        # Code for these analyses would go here.

# Run the analysis function
analyze_test_performance(test_results)
